# medication indicators

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
def lst_ind(dfa_list,ind_val):
    pre0=[]
    for i in dfa_list:
        if pd.isnull(i):
            pre0.append([])
        elif pd.notnull(i):
            si=[]
            jl=i.split(',')
            for ei in jl:
                ef=ei.replace(',','')
                efa,efb,efc=ef.partition(str(ind_val))
                if efa=='':
                    si.append(ef)
            pre0.append(si)
    return pre0

def mm_gen_ind_raw(fid_int,key_code,evnt, detail=False):
    dfc=load_data_by_fid(fid_int)
    #df_icd9m=dfc.copy()
    dfa=dfc.copy()

    dfa_lst=dfa[dfa.columns[1]].values.tolist()
    
    pre0=lst_ind(dfa_lst,str(key_code))
    
    gen_fid_name='fid'+str(fid_int)+'_'+str(evnt)+str(key_code)
    gen_ind_name='ind'+str(fid_int)+'_'+str(evnt)+str(key_code)
    
    dfa[str(gen_fid_name)]=pre0
    dfa[dfa.columns[dfa.columns.get_loc(str(gen_fid_name))]]=dfa[dfa.columns[dfa.columns.get_loc(str(gen_fid_name))]].apply(lambda y: np.nan if len(y)==0 else y )
    
    dfa[str(gen_ind_name)]=pre0
    dfa[dfa.columns[dfa.columns.get_loc(str(gen_ind_name))]]=dfa[dfa.columns[dfa.columns.get_loc(str(gen_ind_name))]].apply(lambda y: 0 if len(y)==0 else 1 )
    
    print('fid '+str(fid_int)+' ',str(evnt)+str(key_code)+' count: '+str(dfa[dfa.columns[dfa.columns.get_loc(str(gen_fid_name))]].count())+', ind '+str(dfa[dfa.columns[dfa.columns.get_loc(str(gen_ind_name))]].count()))
    dfb=dfa[['eid',str(gen_ind_name)]]
    
    if detail==True:
        return dfa
    else:
        return dfb




def mm_gen_ind_list(fid_in, key_code_list, evt, detai=False):
    dfcl=[]
    if detai==False:
        for l in key_code_list:
            df_l=mm_gen_ind_raw(fid_in, l, str(evt), detail=False)
            dfcl.append(df_l)
        dfcl_merge=pd.concat(dfcl,axis=1)
        dfcl_merge=dfcl_merge.loc[:,~dfcl_merge.columns.duplicated()]  # drop duplicated 'eid' columns
        return dfcl_merge

    if detai==True:
        for l in key_code_list:
            df_l=mm_gen_ind_raw(fid_in, l, str(evt), detail=True)
            dfcl.append(df_l)
        dfcl_merge=pd.concat(dfcl,axis=1)
        dfcl_merge=dfcl_merge.loc[:,~dfcl_merge.columns.duplicated()]  # drop duplicated 'eid' columns
        return dfcl_merge

In [3]:
dc4=pd.read_csv('/temp_project/ukbb/data/data_coding/coding4.tsv', sep='\t')

In [4]:
dc4.head()

,coding,meaning
0,1185,vitamin e product [ctsu]
1,1187,chondroitin product
2,1189,co-enzyme q10/ubiquinone/bio-quinone/coenzyme q10
3,1191,indigestion remedy (over the counter)
4,1193,omega-3/fish oil supplement


In [5]:
def search_med_des(keyword):
    klow=str(keyword).lower()
    dc4['meaning']=dc4.meaning.str.lower()
    key_des=dc4[dc4.meaning.str.contains(klow)][['coding','meaning']]
    return key_des

In [6]:
search_med_des('novo')

,coding,meaning
5717,1141168668,novonorm 0.5mg tablet
6122,1141180766,novofem tablet
6608,1141195280,novolizer budesonide 200micrograms/dose cartri...


In [6]:
def related_meds(key_list, dis):
    savepath1='/temp_project/ukbb/data/disease/'
    savepath2=savepath1+str(dis).upper()
    os.chdir(savepath2)
    d_lst=[]
    for k in key_list:
        df_k=search_med_des(str(k).strip())
        d_lst.append(df_k)

    d_coma=pd.concat(d_lst)
    d_comb=d_coma.drop_duplicates()
    print('Searched keyword(s): '+str(key_list)+'\n'+'save related_meds_chk.csv file at '+str(savepath2))
    d_comb.to_csv('related_meds_chk.csv')
    return d_comb


### T1D checked med list: 
praml,syml,apid,exub,hum,humal,humu,ilet,insul

isoph,lantu,lenta,lente,levem,mixtar,monotar,novol,relio,velos

Found possible meds: 

|code|meaning|
|---|---|
|1140883066|insulin product|

### T2D checked med list: 19 meds

|keywords|useful keywords|found code|meanning|
|---|
|appfo,fortam,glucop,glumet,metform,riome|glucop,metform|1140874686|glucophage 500mg tablet
|||1140884600|metformin|
|||1141189090|rosiglitazone 1mg / metformin 500mg tablet|
|janu,ongly,saxagl,sitag,byett,exena,lirag,victoza,nategl,prand,repaglin,starlix|nategl,repaglin,starlix|1141173882|nateglinide|
|||1141168660|repaglinide|
|||1141173786|starlix 60mg tablet|
|acetohex,amary,chlorabeti,chlorr,diabe,diabi,dibat,dymel|acetohex,amary,diabe,diabi|1140857584|acetohexamide|
|glimep,glipiz,glucami,glucotrol,glucovance,glyburi,glycron,glynase|glimep,glipiz|1141156984|amaryl 1mg tablet|
|||1140874736|diabetamide 2.5mg tablet|
|||1140874712|diabinese 100mg tablet|
|||1141152590|glimepiride|
|||1140874646|glipizide|
|||1141157284|glipizide product|
|tolam,insulase,micronase,oribeti,orinas,ronase,tolbutami,tolamid|tolbutami|1140874674|tolbutamide|
|tolazamid,tolinase,actoplus,actos,avand,duetac|tolazamid,actos,avand|1140874664|tolazamide,actos,avand|
|pioglit,rezulin,rosiglitaz,troglitaz|pioglit,rosiglitaz,troglitaz|1141171652|actos 15mg tablet|
|||1141177606|avandia 4mg tablet|
|||1141189094|avandamet 1mg / 500mg tablet|
|||1141171646|pioglitazone|
|||1141177600|rosiglitazone|
|||1141189090|rosiglitazone 1mg / metformin 500mg tablet|
|||1141153254|troglitazone|

useful terms: glucop,metform,nategl,repaglin,starlix,acetohex,amary,diabe,diabi,glimep,glipiz,tolbutami,tolazamid,actos,avand,pioglit,rosiglitaz,troglitaz


In [8]:
meds_T2D=related_meds(input('enter keyword(s) (seperate by ,): ').split(','),input('check disease (T2D/RA/AD): '))
#glucop,metform,nategl,repaglin,starlix,acetohex,amary,diabe,diabi,glimep,glipiz,tolbutami,tolazamid,actos,avand,pioglit,rosiglitaz,troglitaz

print('totsl meds count: '+str(meds_T2D.count()))
meds_T2D

Searched keyword(s): ['glucop', 'metform', 'nategl', 'repaglin', 'starlix', 'acetohex', 'amary', 'diabe', 'diabi', 'glimep', 'glipiz', 'tolbutami', 'tolazamid', 'actos', 'avand', 'pioglit', 'rosiglitaz', 'troglitaz']
save related_meds_chk.csv file at /temp_project/ukbb/data/disease/T2D
totsl meds count: coding     21
meaning    21
dtype: int64


,coding,meaning
2928,1140874686,glucophage 500mg tablet
4206,1140884600,metformin
6420,1141189090,rosiglitazone 1mg / metformin 500mg tablet
5942,1141173882,nateglinide
5716,1141168660,repaglinide
5938,1141173786,starlix 60mg tablet
734,1140857584,acetohexamide
5355,1141156984,amaryl 1mg tablet
2938,1140874736,diabetamide 2.5mg tablet
2931,1140874712,diabinese 100mg tablet


In [9]:
meds_T2D_list=meds_T2D.coding.values.tolist()

In [10]:
meds_T2D_list[0:5]

[1140874686, 1140884600, 1141189090, 1141173882, 1141168660]

In [11]:
os.chdir('/temp_project/all_codes')
from ukbb_ldbf import load_data_by_fid

In [12]:
meds=load_data_by_fid(fid=20003)

fid 20003 is a single-measure categorical (multiple) variable, which is 
Treatment/medication codeUses data-coding 4 comprises 6745 Integer-valued members in a simple list.


In [13]:
meds.vec_cam20003i0.dropna().count()

364052

In [16]:

meds_ind_pre=mm_gen_ind_list(fid_in=20003, key_code_list=meds_T2D_list, evt='T2D')

fid 20003 is a multiple-measure categorical (multiple) variable
fid 20003  T2D1140874686 count: 492, ind 502602
fid 20003 is a multiple-measure categorical (multiple) variable
fid 20003  T2D1140884600 count: 13995, ind 502602
fid 20003 is a multiple-measure categorical (multiple) variable
fid 20003  T2D1141189090 count: 214, ind 502602
fid 20003 is a multiple-measure categorical (multiple) variable
fid 20003  T2D1141173882 count: 42, ind 502602
fid 20003 is a multiple-measure categorical (multiple) variable
fid 20003  T2D1141168660 count: 84, ind 502602
fid 20003 is a multiple-measure categorical (multiple) variable
fid 20003  T2D1141173786 count: 8, ind 502602
fid 20003 is a multiple-measure categorical (multiple) variable
fid 20003  T2D1140857584 count: 0, ind 502602
fid 20003 is a multiple-measure categorical (multiple) variable
fid 20003  T2D1141156984 count: 49, ind 502602
fid 20003 is a multiple-measure categorical (multiple) variable
fid 20003  T2D1140874736 count: 0, ind 502602

In [22]:

df_ind20003_T2D=meds_ind_pre[['eid']].copy()
df_ind20003_T2D['ind20003_T2D']=meds_ind_pre.drop('eid',axis=1).sum(axis=1)
df_ind20003_T2D.ind20003_T2D=df_ind20003_T2D.ind20003_T2D.apply(lambda y: 1 if int(y)>0 else 0)
df_ind20003_T2D.ind20003_T2D.value_counts()

0    487267
1     15335
Name: ind20003_T2D, dtype: int64

In [25]:
os.chdir('/temp_project/ukbb/data/i0/varOutcome/T2D')
df_ind20003_T2D.to_csv('meds_ind_T2D.csv', index=None)

In [28]:
df_ind20003_possibleT1D=mm_gen_ind_list(fid_in=20003, key_code_list=['1140883066'], evt='T2D')

fid 20003 is a multiple-measure categorical (multiple) variable
fid 20003  T2D1140883066 count: 5317, ind 502602


In [22]:
#os.mkdir('/temp_project/ukbb/data/disease/DM')

In [24]:
meds_insulin=related_meds(input('enter keyword(s) (seperate by ,): ').split(','),input('check disease (DM/T2D/RA/AD): '))
# insulin

print('totsl meds count: '+str(meds_insulin.coding.count()))
meds_insulin

Searched keyword(s): ['insulin']
save related_meds_chk.csv file at /temp_project/ukbb/data/disease/DM
totsl meds count: 1


,coding,meaning
3999,1140883066,insulin product


In [25]:
meds_insulin_list=meds_insulin.coding.values.tolist()

In [26]:

insulin_meds_ind_pre=mm_gen_ind_list(fid_in=20003, key_code_list=meds_insulin_list, evt='DM')

fid 20003 is a multiple-measure categorical (multiple) variable
fid 20003  DM1140883066 count: 5317, ind 502602


In [27]:
insulin_meds_ind_pre.columns=['eid','ind20003_insulin']

In [29]:
insulin_meds_ind_pre.ind20003_insulin.value_counts()

0    497285
1      5317
Name: ind20003_insulin, dtype: int64

In [31]:
insulin_meds_ind_pre.to_csv('/temp_project/ukbb/data/i0/varOutcome/T2D/meds_ind_insulin.csv', index=None)